In [1]:
import pandas as pd
import numpy as np

# Configuration
csv_path = "fetch_with_id_log.csv"
train_ratio = 0.8
val_ratio = 0.1
# Test ratio is the remainder (0.1)

df = pd.read_csv(csv_path)

# Ensure strict string type for IDs to match filenames later
df["id"] = df["id"].astype(str)

# Create a new column for the split
df["split"] = None

# Process each MBTI category separately to ensure class balance
mbti_categories = df["four_letter"].unique()

for mbti in mbti_categories:
    # 1. Get all unique celebrity IDs for this specific MBTI
    # (We split by ID, not by image, to prevent leakage)
    unique_ids = df[df["four_letter"] == mbti]["id"].unique()

    # 2. Shuffle them
    np.random.shuffle(unique_ids)

    # 3. Calculate split points
    n_ids = len(unique_ids)
    n_train = int(n_ids * train_ratio)
    n_val = int(n_ids * val_ratio)

    # 4. Slice the array of IDs
    train_ids = unique_ids[:n_train]
    val_ids = unique_ids[n_train : n_train + n_val]
    test_ids = unique_ids[n_train + n_val :]

    # 5. Assign the split label to the main dataframe
    df.loc[df["id"].isin(train_ids), "split"] = "train"
    df.loc[df["id"].isin(val_ids), "split"] = "val"
    df.loc[df["id"].isin(test_ids), "split"] = "test"

# Save the updated log
df.to_csv("fetch_log_with_splits.csv", index=False)
print("Updated log saved to 'fetch_log_with_splits.csv'.")
print(df["split"].value_counts())

Updated log saved to 'fetch_log_with_splits.csv'.
split
train    2530
test      319
val       314
Name: count, dtype: int64


In [2]:
import os
import shutil
import pandas as pd
from pathlib import Path
from tqdm import tqdm

import torch
from PIL import Image
from ultralytics import YOLO
import pandas as pd

MODEL_WEIGHTS = "yolov12l-face.pt"  # path to face model weights
INPUT_ROOT = Path("images_with_id")  # your existing images/{MBTI}/...
OUTPUT_ROOT = Path("faces_yolo_id_split_384")  # where cropped faces go
CONF_THRESH = 0.25  # min confidence for a face
TARGET_SIZE = 384  # final face size (square)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"[INFO] Loading model {MODEL_WEIGHTS} on {DEVICE}")
model = YOLO(MODEL_WEIGHTS)
model.to(DEVICE)

records = []
yolo_image_id = 0

[INFO] Loading model yolov12l-face.pt on cuda


In [3]:
def load_image(path: Path) -> Image.Image | None:
    try:
        return Image.open(path).convert("RGB")
    except Exception as e:
        print(f"[WARN] Cannot open {path}: {e}")
        return None

def process_one_image(img_path: Path, out_path: Path, mbti_class) -> Image.Image | None:
    global yolo_image_id
    rel_in = img_path.relative_to(INPUT_ROOT)
    pil_img = load_image(img_path)
    if pil_img is None:
        return None

    # convert to numpy (RGB)
    img_np = np.array(pil_img)

    # run YOLO-face; returns list[Results], we take first
    results = model.predict(img_np, verbose=False, conf=CONF_THRESH, device=DEVICE)
    if not results:
        print(f"[SKIP] No results for {rel_in}")
        return None

    res = results[0]
    boxes = res.boxes  # Boxes object: xyxy, conf, etc.
    if boxes is None or len(boxes) == 0:
        print(f"[SKIP] No faces above conf={CONF_THRESH} in {rel_in}")
        return None

    xyxy = boxes.xyxy.cpu().numpy()  # (N, 4)
    conf = boxes.conf.cpu().numpy()  # (N,)

    # choose largest box by area
    areas = (xyxy[:, 2] - xyxy[:, 0]) * (xyxy[:, 3] - xyxy[:, 1])
    best_idx = int(areas.argmax())

    x1, y1, x2, y2 = xyxy[best_idx]
    x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
    c = float(conf[best_idx])

    # basic sanity
    if x2 <= x1 or y2 <= y1:
        print(f"[SKIP] Degenerate box in {rel_in}")
        return None

    # crop + resize face
    face = pil_img.crop((x1, y1, x2, y2))
    face = face.resize((TARGET_SIZE, TARGET_SIZE), Image.BICUBIC)

    face.save(out_path)

    rel_out = out_path.relative_to(OUTPUT_ROOT)
    records.append(
        {
            "id": yolo_image_id,
            "mbti": mbti_class,
            "input_path": str(rel_in).replace("\\", "/"),
            "face_path": str(rel_out).replace("\\", "/"),
            "conf": c,
            "bbox_x1": x1,
            "bbox_y1": y1,
            "bbox_x2": x2,
            "bbox_y2": y2,
        }
    )
    yolo_image_id += 1
    print(f"[OK] {rel_in} -> {rel_out} (conf={c:.3f})")

In [4]:
# --- Config ---
# Path to the NEW csv generated in Snippet 1
csv_path = "fetch_log_with_splits.csv"
source_root = INPUT_ROOT
target_root = OUTPUT_ROOT

# --- Load Lookup Table ---
print("Loading split mapping...")
df = pd.read_csv(csv_path)
id_to_split = df.set_index(df["id"].astype(str))["split"].to_dict()

# --- Processing ---
# We iterate through the existing MBTI folders in the source
mbti_dirs = [d for d in source_root.iterdir() if d.is_dir()]

for mbti_dir in tqdm(mbti_dirs, desc="Processing MBTI classes"):
    mbti_class = mbti_dir.name

    for img_path in mbti_dir.glob("*.jpg"):
        filename = img_path.name

        # Filename format is: {id}_{hexdigest}.jpg
        # We extract the ID
        try:
            img_id = filename.split("_")[0]
        except IndexError:
            print(f"Skipping malformed filename: {filename}")
            continue

        # Lookup the split
        split = id_to_split.get(img_id)

        if split is None:
            split = "train"

        # Construct Destination Path:
        # faces_yolo_annotated / train / INTJ / {id}_{hex}.jpg
        dest_dir = target_root / split / mbti_class
        dest_dir.mkdir(parents=True, exist_ok=True)

        out_name = img_path.stem + "_face.jpg"
        dest_path = dest_dir / out_name

        process_one_image(img_path, dest_path, mbti_class)


Loading split mapping...


Processing MBTI classes:   0%|          | 0/16 [00:00<?, ?it/s]

[OK] ENFJ\10153_3fcd1856b1.jpg -> train\ENFJ\10153_3fcd1856b1_face.jpg (conf=0.896)
[OK] ENFJ\10153_60ddb3affb.jpg -> train\ENFJ\10153_60ddb3affb_face.jpg (conf=0.910)
[OK] ENFJ\10153_88240705a6.jpg -> train\ENFJ\10153_88240705a6_face.jpg (conf=0.908)
[OK] ENFJ\10153_a492d7f160.jpg -> train\ENFJ\10153_a492d7f160_face.jpg (conf=0.910)
[OK] ENFJ\102806_05ecb10c63.jpg -> train\ENFJ\102806_05ecb10c63_face.jpg (conf=0.899)
[OK] ENFJ\102806_26eae27c79.jpg -> train\ENFJ\102806_26eae27c79_face.jpg (conf=0.904)
[OK] ENFJ\102806_54df924316.jpg -> train\ENFJ\102806_54df924316_face.jpg (conf=0.885)
[OK] ENFJ\102806_d4f8136328.jpg -> train\ENFJ\102806_d4f8136328_face.jpg (conf=0.896)
[OK] ENFJ\10294_3b221e3557.jpg -> train\ENFJ\10294_3b221e3557_face.jpg (conf=0.894)
[OK] ENFJ\10294_8887601966.jpg -> train\ENFJ\10294_8887601966_face.jpg (conf=0.930)
[OK] ENFJ\10294_c06d3d4e3f.jpg -> train\ENFJ\10294_c06d3d4e3f_face.jpg (conf=0.914)
[OK] ENFJ\10294_fe38ced4d8.jpg -> train\ENFJ\10294_fe38ced4d8_face.j

Processing MBTI classes:   6%|▋         | 1/16 [00:31<07:50, 31.40s/it]

[OK] ENFJ\995_cfc1e7e2be.jpg -> train\ENFJ\995_cfc1e7e2be_face.jpg (conf=0.899)
[OK] ENFP\10257_1f4100b291.jpg -> train\ENFP\10257_1f4100b291_face.jpg (conf=0.895)
[OK] ENFP\10257_53d7e317c0.jpg -> train\ENFP\10257_53d7e317c0_face.jpg (conf=0.913)
[OK] ENFP\10257_9d268391e0.jpg -> train\ENFP\10257_9d268391e0_face.jpg (conf=0.890)
[OK] ENFP\10257_d2f78d1a67.jpg -> train\ENFP\10257_d2f78d1a67_face.jpg (conf=0.885)
[OK] ENFP\103932_7ccde5091f.jpg -> val\ENFP\103932_7ccde5091f_face.jpg (conf=0.862)
[OK] ENFP\103932_8a6d306c1c.jpg -> val\ENFP\103932_8a6d306c1c_face.jpg (conf=0.900)
[OK] ENFP\103932_8e7fd77981.jpg -> val\ENFP\103932_8e7fd77981_face.jpg (conf=0.887)
[OK] ENFP\103932_acc9cc09bf.jpg -> val\ENFP\103932_acc9cc09bf_face.jpg (conf=0.886)
[OK] ENFP\105487_06a799c5c0.jpg -> train\ENFP\105487_06a799c5c0_face.jpg (conf=0.930)
[OK] ENFP\105487_610ac44cdf.jpg -> train\ENFP\105487_610ac44cdf_face.jpg (conf=0.897)
[OK] ENFP\105487_ac57124835.jpg -> train\ENFP\105487_ac57124835_face.jpg (co

...

[OK] ISTJ\10027_548761e67b.jpg -> test\ISTJ\10027_548761e67b_face.jpg (conf=0.890)
[OK] ISTJ\10027_c4c03872c5.jpg -> test\ISTJ\10027_c4c03872c5_face.jpg (conf=0.905)
[OK] ISTJ\10027_e00c42aacd.jpg -> test\ISTJ\10027_e00c42aacd_face.jpg (conf=0.894)
[OK] ISTJ\10027_e558c2e5a3.jpg -> test\ISTJ\10027_e558c2e5a3_face.jpg (conf=0.873)
[OK] ISTJ\10032_390bad343f.jpg -> train\ISTJ\10032_390bad343f_face.jpg (conf=0.903)
[OK] ISTJ\10032_8ff51f4f16.jpg -> train\ISTJ\10032_8ff51f4f16_face.jpg (conf=0.890)
[OK] ISTJ\10032_c24126af1a.jpg -> train\ISTJ\10032_c24126af1a_face.jpg (conf=0.904)
[OK] ISTJ\102250_0444164cbb.jpg -> train\ISTJ\102250_0444164cbb_face.jpg (conf=0.893)
[OK] ISTJ\102250_6754e55d36.jpg -> train\ISTJ\102250_6754e55d36_face.jpg (conf=0.873)
[OK] ISTJ\102250_e886bbc4c6.jpg -> train\ISTJ\102250_e886bbc4c6_face.jpg (conf=0.910)
[OK] ISTJ\102250_f77f680dd9.jpg -> train\ISTJ\102250_f77f680dd9_face.jpg (conf=0.898)
[OK] ISTJ\10253_5d34cf7796.jpg -> train\ISTJ\10253_5d34cf7796_face.jpg (

Processing MBTI classes:  94%|█████████▍| 15/16 [12:47<00:47, 47.49s/it]

[OK] ISTP\100515_4948531c73.jpg -> train\ISTP\100515_4948531c73_face.jpg (conf=0.902)
[OK] ISTP\100515_5388bd41c2.jpg -> train\ISTP\100515_5388bd41c2_face.jpg (conf=0.905)
[OK] ISTP\100515_ba73ad82ef.jpg -> train\ISTP\100515_ba73ad82ef_face.jpg (conf=0.934)
[OK] ISTP\100515_cfbde75405.jpg -> train\ISTP\100515_cfbde75405_face.jpg (conf=0.903)
[OK] ISTP\101404_2a6f563004.jpg -> test\ISTP\101404_2a6f563004_face.jpg (conf=0.876)
[OK] ISTP\101404_4dc2b1a3c6.jpg -> test\ISTP\101404_4dc2b1a3c6_face.jpg (conf=0.906)
[OK] ISTP\101404_7dc648de97.jpg -> test\ISTP\101404_7dc648de97_face.jpg (conf=0.886)
[OK] ISTP\101404_e2cf4137ae.jpg -> test\ISTP\101404_e2cf4137ae_face.jpg (conf=0.920)
[OK] ISTP\10157_579c621891.jpg -> train\ISTP\10157_579c621891_face.jpg (conf=0.923)
[OK] ISTP\10157_99a72ce24b.jpg -> train\ISTP\10157_99a72ce24b_face.jpg (conf=0.880)
[OK] ISTP\10157_eeafa18daa.jpg -> train\ISTP\10157_eeafa18daa_face.jpg (conf=0.878)
[OK] ISTP\102059_3fa2662ec9.jpg -> train\ISTP\102059_3fa2662ec9_

Processing MBTI classes: 100%|██████████| 16/16 [13:16<00:00, 49.78s/it]

[OK] ISTP\9955_e4575162aa.jpg -> train\ISTP\9955_e4575162aa_face.jpg (conf=0.910)
[OK] ISTP\9955_f09f554cb6.jpg -> train\ISTP\9955_f09f554cb6_face.jpg (conf=0.914)
[OK] ISTP\99835_62fa7dace2.jpg -> train\ISTP\99835_62fa7dace2_face.jpg (conf=0.902)
[OK] ISTP\99835_81f6967c44.jpg -> train\ISTP\99835_81f6967c44_face.jpg (conf=0.888)
[OK] ISTP\99835_b5561a0a56.jpg -> train\ISTP\99835_b5561a0a56_face.jpg (conf=0.904)
[OK] ISTP\99835_d59a17c271.jpg -> train\ISTP\99835_d59a17c271_face.jpg (conf=0.787)


In [5]:
if records:
    df = pd.DataFrame(records)
    df.to_csv("faces_yolo_id_split_384_metadata.csv", index=False)
    print(f"\n[DONE] Saved {len(records)} face crops.")
else:
    print("\n[WARN] No faces saved.")


[DONE] Saved 11979 face crops.
